# Valuing Caps and Floors with QL Example

We value caps and floors using Black's model and try to replicate a QL example at 
http://gouthamanbalaraman.com/blog/interest-rate-cap-floor-valuation-quantlib-python.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.market.curves import *

####################################################################
#  FINANCEPY BETA Version 0.350 - This build: 30 Apr 2024 at 21:20 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Building a Ibor Discount Curve

The example does this using a set of dates and zero ratesm

In [3]:
value_dt = Date(14, 6, 2016)

We have a vector of dates and zero rates

In [4]:
dates = [Date(14,6,2016), Date(14,9,2016), Date(14,12,2016), Date(14,6,2017),
         Date(14,6,2019), Date(14,6,2021), Date(15,6,2026), Date(16,6,2031),
         Date(16,6,2036), Date(14,6,2046)]
rates = [0.000000, 0.006616, 0.007049, 0.007795, 0.009599, 0.011203, 0.015068, 0.017583,
         0.018998, 0.020080]

frequencyType = FrequencyTypes.ANNUAL
dc_type = DayCountTypes.ACT_ACT_ISDA

In [5]:
discount_curve = DiscountCurveZeros(value_dt, dates, rates, frequencyType, 
                                      dc_type, InterpTypes.LINEAR_ZERO_RATES)

In [6]:
print(discount_curve)

OBJECT TYPE: DiscountCurveZeros
VALUATION DATE: 14-JUN-2016
FREQUENCY TYPE: FrequencyTypes.ANNUAL
DAY COUNT TYPE: DayCountTypes.ACT_ACT_ISDA
INTERP TYPE: InterpTypes.LINEAR_ZERO_RATES
DATES: ZERO RATES
 14-JUN-2016:  0.0000000
 14-SEP-2016:  0.0066160
 14-DEC-2016:  0.0070490
 14-JUN-2017:  0.0077950
 14-JUN-2019:  0.0095990
 14-JUN-2021:  0.0112030
 15-JUN-2026:  0.0150680
 16-JUN-2031:  0.0175830
 16-JUN-2036:  0.0189980
 14-JUN-2046:  0.0200800



In [7]:
start_dt = Date(14, 6, 2016)
end_dt = Date(14, 6 , 2026)
cal_type = CalendarTypes.UNITED_STATES
bd_type = BusDayAdjustTypes.MODIFIED_FOLLOWING
frequencyType = FrequencyTypes.QUARTERLY
dg_type = DateGenRuleTypes.FORWARD
last_fixing = 0.0065560
notional = 1000000
dc_type=DayCountTypes.ACT_360
option_type = FinCapFloorTypes.CAP
strike_rate = 0.02

In [8]:
cap = IborCapFloor(start_dt, end_dt, option_type, strike_rate, last_fixing, 
                       frequencyType,  dc_type, notional,
                       cal_type, bd_type, dg_type)

In [9]:
print(cap)

OBJECT TYPE: IborCapFloor
START DATE: 14-JUN-2016
MATURITY DATE: 14-JUN-2026
STRIKE COUPON: 2.0
OPTION TYPE: FinCapFloorTypes.CAP
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.ACT_360


## Valuation

### Black's Model

In [10]:
blackVol = 0.547295
model = Black(blackVol)

In [11]:
cap.value(value_dt, discount_curve, model)

54416.622944954404

This is very close to QL which finds 54,369.858

In [12]:
cap.print_leg()

START DATE: 14-JUN-2016
MATURITY DATE: 14-JUN-2026
OPTION TYPE FinCapFloorTypes.CAP
STRIKE (%): 2.0
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.ACT_360
VALUATION DATE 14-JUN-2016
PAYMENT_dt     YEAR_FRAC   FWD_RATE    INTRINSIC           DF    CAPLET_PV       CUM_PV
    14-JUN-2016          -         -            -     1.000000            -            -
    14-SEP-2016  0.2555556   0.65560         0.00     0.998344         0.00         0.00
    14-DEC-2016  0.2527778   0.73436         0.00     0.996494         0.03         0.03
    14-MAR-2017  0.2500000   0.80253         0.00     0.994499         3.73         3.75
    14-JUN-2017  0.2555556   0.87618         0.00     0.992277        25.56        29.31
    14-SEP-2017  0.2555556   0.87712         0.00     0.990058        51.60        80.91
    14-DEC-2017  0.2527778   0.92143         0.00     0.987757        98.44       179.35
    14-MAR-2018  0.2500000   0.96525         0.00     0.985379       155.99       335.34
    

Copyright (c) 2020 Dominic O'Kane

This value is within a few dollars of the Quantlib valuation.